In [2]:
#from test_env import TestEnv as TestEnv_2
import gym
#import gymnasium as gym
import pygame
from algorithms.rl import RL

from environment.algorithms_rl import RL as RL_2
from environment.algorithms_planner import Planner as Planner_2

from examples.test_env import TestEnv
from environment.test_env import TestEnv_2 as TestEnv_2
import environment.frozen_lake as FL
from algorithms.planner import Planner


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from matplotlib.colors import LinearSegmentedColormap

import environment.multi_ as ml5
from itertools import repeat
import json

#from gym.envs.toy_text.frozen_lake import generate_random_map

import pickle
import awkward as ak

# Function definitions

## Model evaluation function and decretizers : Policy and value iteratios

In [251]:

        
        
def mapper_state(state, seriaized):
        return np.where(
(seriaized == state).sum(axis =1) == 2)[0][0]


def discretize_state(state,low,bin_width,state_space):
        
        
        
        state_adj = low + bin_width*((state - low)//bin_width)
        map_ = state_space[np.all(state_space.astype('float16') ==  (state_adj).astype('float16'),axis=2)]
        #print(state)
        state_adj = map_[0]
        return state_adj



def Model_evaluations_tests(n_="VI_FL_size_12_gamma_0.99_theta_1e-13",
                            env='FL',algo='Valu_iter',
                           n_2 = None,
                           
                           render=False,user_input=False ,n_iters=50,
                            
                            interval = 10, sel_select = True, sel = 1000,
                            
                            version = 1,
                           
                           additional_info = None
                           
                           
                           ):


    test_score_list_all = []
    test_steps_list_all = []
    time_s_list_all = []



    #n_ = "VI_FL_size_12_gamma_0.99_theta_1e-13"
    addr = "ADA/"+env+"/"+algo

    
    
    if version == 1:
        n_env_map = n_
    else:
        n_env_map = n_2
    
    env_list = pd.read_pickle(
            addr+"/env_list/"+n_env_map+".gzip"
    ,compression ='gzip')
    
    

    if  env == 'MC':
        
        env_list
        
        #print(env_list)
        
        env_list_base = env_list[0]
        env_list = [i.env for i in env_list[0].values]
        
        
        packages = pickle.load(open('environment/MountainCar_envP_config.pkl', "rb"))

        package_select = packages[additional_info['size']]
        
        
        def _convert_state_obs_method(state,done,package_select=package_select):
        
        
        
        
            return (-1 if done else mapper_state(
                                        discretize_state(state,
                                                 package_select['low'],
                                                package_select['bin_width'],
                                                package_select['state_space']),
                                        package_select['seriaized']))
        
        
        

        
        
        
        
    else:
        env_list = env_list[0]

    p_list = pd.read_pickle(
            addr+"/other_info/P/"+n_+".gzip"
    ,compression ='gzip')





    for j in range(p_list.shape[0]):#responses['train_responses'])):

        test_score_list =[]
        test_steps_list =[]
        time_s_list =[]

        if sel_select:
            sel = p_list.optima[j]


        #def 

        final_pi_list = pd.read_pickle(
            addr+"/PI_track/"+n_+"_"+str(j)+".gzip"
    ,compression ='gzip')

        final_pi_list.drop('trial',axis =1,inplace = True)



        for i in range(0,sel-1,interval):#final_pi_list.shape[0],1000):



            if i>=final_pi_list.shape[0]:
                print(i,final_pi_list.shape,j)


            pi_1  = final_pi_list.iloc[i].astype(int).to_dict()#.values


            
            
            
            
            if env == 'MC':
            


                test_scores,test_steps,time_s = TestEnv_2.test_env(env=
                                                env_list[j],
                                                #frozen_lake.env, 
                                               #parellel_func = ml5.driver_func,
                                                            render=render, 
                                                user_input=user_input, pi=pi_1#pi
                                                 #,PROCESSES = 8,
                                                 #type_ = 'starmap'
                                                 ,n_iters=n_iters
                                                                   
                                                                   
                                                  ,convert_state_obs=_convert_state_obs_method#env_list_base[j].convert_state_obs               

                                                           )
            else:
                test_scores,test_steps,time_s = TestEnv_2.test_env(env=
                                                env_list[j],
                                                #frozen_lake.env, 
                                               #parellel_func = ml5.driver_func,
                                                            render=render, 
                                                user_input=user_input, pi=pi_1#pi
                                                 #,PROCESSES = 8,
                                                 #type_ = 'starmap'
                                                 ,n_iters=n_iters
                                                                   
                                                                                  

                                                           )                
                
                
            env_list[j].close()
            test_score_list.append(test_scores.mean())
            test_steps_list.append(test_steps.mean())
            time_s_list.append(time_s.mean())
        test_score_list_all.append(test_score_list)
        test_steps_list_all.append(test_steps_list)
        time_s_list_all.append(time_s_list)



    test_score_list_all_df = pd.DataFrame(ak.fill_none(ak.pad_none(test_score_list_all, p_list.optima.max(), axis=-1), np.nan).to_numpy()).T.ffill()
    test_steps_list_all_df = pd.DataFrame(ak.fill_none(ak.pad_none(test_steps_list_all, p_list.optima.max(), axis=-1), np.nan).to_numpy()).T.ffill()
    time_s_list_all_df = pd.DataFrame(ak.fill_none(ak.pad_none(time_s_list_all , p_list.optima.max(), axis=-1), np.nan).to_numpy()).T.ffill()
    
    
    
    
    test_score_list_all_df.to_pickle(
        
        addr+"/evaluation/score/"+n_+
        
        '.gzip',compression='gzip')
    test_steps_list_all_df.to_pickle(addr+"/evaluation/steps/"+n_+
        
        '.gzip',compression='gzip')
    time_s_list_all_df.to_pickle(addr+"/evaluation/time/"+n_+
        
        '.gzip',compression='gzip')
    
 

## Model evaluation function and decretizers : Q learning

In [ ]:
import awkward as ak



def mapper_state(state, seriaized):
        return np.where(
(seriaized == state).sum(axis =1) == 2)[0][0]


def discretize_state(state,low,bin_width,state_space):
        
        
        
        state_adj = low + bin_width*((state - low)//bin_width)
        map_ = state_space[np.all(state_space.astype('float16') ==  (state_adj).astype('float16'),axis=2)]
        #print(state)
        state_adj = map_[0]
        return state_adj





def Model_evaluations_tests_Q(n_="QL_FL_size_4_gamma_0.9_alpha_0.95_eps_0.4.gzip",
                            env='FL',algo='RL_iter',
                           n_2 = None,
                           
                           render=False,user_input=False ,n_iters=50,
                            
                            interval = 100, sel_select = True, sel = 1000,
                           
                            version = 1,
                           
                           additional_info = None
                           
                           
                           
                           
                           ):


    test_score_list_all = []
    test_steps_list_all = []
    time_s_list_all = []



    #n_ = "VI_FL_size_12_gamma_0.99_theta_1e-13"
    addr = "ADA/"+env+"/"+algo


    if version == 1:
        n_env_map = n_
    else:
        n_env_map = n_2




    env_list = pd.read_pickle(
            addr+"/env_list/"+n_env_map+".gzip"
    ,compression ='gzip')
    
    



    if  env == 'MC':
        
        env_list
        
        #print(env_list)
        
        env_list_base = env_list[0]
        env_list = [i.env for i in env_list[0].values]
        
        
        packages = pickle.load(open('environment/MountainCar_envP_config.pkl', "rb"))

        package_select = packages[additional_info['size']]
        
        
        def _convert_state_obs_method(state,done,package_select=package_select):
        
        
        
        
            return (-1 if done else mapper_state(
                                        discretize_state(state,
                                                 package_select['low'],
                                                package_select['bin_width'],
                                                package_select['state_space']),
                                        package_select['seriaized']))
        
        
        

        
        
        
        
    else:
        env_list = env_list[0]



















    p_list = pd.read_pickle(
            addr+"/other_info/P/"+n_+".gzip"
    ,compression ='gzip')





    for j in range(p_list.shape[0]):#responses['train_responses'])):

        test_score_list =[]
        test_steps_list =[]
        time_s_list =[]

        if sel_select:
            sel = p_list.optima[j]


        #def 

        final_pi_list = pd.read_pickle(
            addr+"/PI_track/"+n_+"_"+str(j)+".gzip"
    ,compression ='gzip')

        final_pi_list.drop('trial',axis =1,inplace = True)



        for i in range(0,sel-1,interval):#final_pi_list.shape[0],1000):



            if i>=final_pi_list.shape[0]:
                print(i,final_pi_list.shape,j)


            pi_1  = final_pi_list.iloc[i].astype(int).to_dict()#.values








            if env == 'MC':
            


                test_scores,test_steps,time_s = TestEnv_2.test_env(env=
                                                env_list[j],
                                                #frozen_lake.env, 
                                               #parellel_func = ml5.driver_func,
                                                            render=render, 
                                                user_input=user_input, pi=pi_1#pi
                                                 #,PROCESSES = 8,
                                                 #type_ = 'starmap'
                                                 ,n_iters=n_iters
                                                                   
                                                                   
                                                  ,convert_state_obs=_convert_state_obs_method#env_list_base[j].convert_state_obs               

                                                           )
            else:
                test_scores,test_steps,time_s = TestEnv_2.test_env(env=
                                                env_list[j],
                                                #frozen_lake.env, 
                                               #parellel_func = ml5.driver_func,
                                                            render=render, 
                                                user_input=user_input, pi=pi_1#pi
                                                 #,PROCESSES = 8,
                                                 #type_ = 'starmap'
                                                 ,n_iters=n_iters
                                                                   
                                                                                  

                                                           )  
            env_list[j].close()
            test_score_list.append(test_scores.mean())
            test_steps_list.append(test_steps.mean())
            time_s_list.append(time_s.mean())
        test_score_list_all.append(test_score_list)
        test_steps_list_all.append(test_steps_list)
        time_s_list_all.append(time_s_list)


    # i have created an issue here ...... damit it should be padded to only optima.max/interval ... but it is optima ... 
    test_score_list_all_df = pd.DataFrame(ak.fill_none(ak.pad_none(test_score_list_all, p_list.optima.max(), axis=-1), np.nan).to_numpy()).T.ffill()
    test_steps_list_all_df = pd.DataFrame(ak.fill_none(ak.pad_none(test_steps_list_all, p_list.optima.max(), axis=-1), np.nan).to_numpy()).T.ffill()
    time_s_list_all_df = pd.DataFrame(ak.fill_none(ak.pad_none(time_s_list_all , p_list.optima.max(), axis=-1), np.nan).to_numpy()).T.ffill()


    
    test_score_list_all_df.to_pickle(
        
        addr+"/evaluation/score/"+n_+
        
        '.gzip',compression='gzip')
    test_steps_list_all_df.to_pickle(addr+"/evaluation/steps/"+n_+
        
        '.gzip',compression='gzip')
    time_s_list_all_df.to_pickle(addr+"/evaluation/time/"+n_+
        
        '.gzip',compression='gzip')
    
    #plt.plot(np.array(time_s_list_all).mean(axis =0))
    #plt.plot(np.array(test_score_list_all).mean(axis =0),marker = '.')
    #plt.plot(np.array(test_steps_list_all).mean(axis =0))  
    #    

# Evaluations

## Mountain car Policy Iteration evaluatons | version 2

In [252]:
pd.read_csv('PI_MCMCpolic_iter2.csv').apply(
lambda x:

                    Model_evaluations_tests(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                           n_2 = x['misc']+'_size_'+str(x['size'])
                       
                        ,version = 2,
                           render=False,user_input=False ,n_iters=50,
                            
                            interval = 1, sel_select = True, sel = 1000,
                                            additional_info = {'size':x['size']}
                           
                           
                           
                       
                       )

,axis =1
)


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
dtype: object

## Mountain car Policy Iteration evaluatons | version 1

In [256]:
pd.read_csv('VI_MCMCValu_iter1.csv').apply(
lambda x:

                    Model_evaluations_tests(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                           n_2 = x['misc']+'_size_'+str(x['size'])
                       
                        ,version = 2,
                           render=False,user_input=False ,n_iters=50,
                            
                            interval = 10, sel_select = True, sel = 1000,
                                            additional_info = {'size':x['size']}
                           
                           
                           
                       
                       )

,axis =1
)


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
dtype: object

In [211]:
addr = 'ADA/MC/polic_iter/env_list'#"ADA/"+'MC'+"/"+'polic_iter'


ad_ = "ADA/MC/polic_iter/env_list/PI_MC_size_15.gzip"

env_list = pd.read_pickle(
ad_#addr+"/"+"PI_FL_size_4_gamma_0.01_theta_1e-13"+".gzip"
,compression ='gzip',

#version = 2

)

## Frozen Lake Policy Iteration evaluatons | version 1 and version 2

In [127]:

pd.read_csv('PI_FLFLpolic_iter1.csv').apply(
lambda x:

Model_evaluations_tests(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                       
                       
                        
                           render=False,user_input=False ,n_iters=50,
                            
                            interval = 1, sel_select = True, sel = 1000
                           
                           
                           
                       
                       )

,axis =1
)




pd.read_csv('PI_FLFLpolic_iter2.csv').apply(
lambda x:

Model_evaluations_tests(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                       
                       
                        
                           render=False,user_input=False ,n_iters=50,
                            
                            interval = 1, sel_select = True, sel = 1000
                           
                           
                           
                       
                       )

,axis =1
)





/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
dtype: object

## Frozen Lake Value Iteration evaluatons | version 1 and version 2

In [128]:

pd.read_csv('VI_FLFLValu_iter1.csv').apply(
lambda x:

Model_evaluations_tests(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                       
                       
                        
                           render=False,user_input=False ,n_iters=50,
                            
                            interval = 10, sel_select = True, sel = 1000
                           
                           
                           
                       
                       )

,axis =1
)




pd.read_csv('VI_FLFLValu_iter2.csv').apply(
lambda x:

Model_evaluations_tests(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                       
                       
                        
                           render=False,user_input=False ,n_iters=50,
                            
                            interval = 10, sel_select = True, sel = 1000
                           
                           
                           
                       
                       )

,axis =1
)





/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
dtype: object

## Moutain car Q learning evaluatons | version 10

In [370]:
pd.read_csv('QL_MCMCRL_iter10.csv').apply(
lambda x:

                    Model_evaluations_tests_Q(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                           n_2 = x['misc']+'_size_'+str(x['size'])
                       
                        ,version = 2,
                           render=False,user_input=False ,n_iters=10,
                            
                            interval = 10, sel_select = True, sel = 1000,
                                            additional_info = {'size':x['size']}
                           
                           
                           
                       
                       )

,axis =1
)


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

0      None
1      None
2      None
3      None
4      None
       ... 
115    None
116    None
117    None
118    None
119    None
Length: 120, dtype: object

## Moutain car Q learning evaluatons | version 2

In [457]:
pd.read_csv('QL_MCMCRL_iter2.csv').apply(
lambda x:

                    Model_evaluations_tests_Q(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                           n_2 = x['misc']+'_size_'+str(x['size'])
                       
                        ,version = 2,
                           render=False,user_input=False ,n_iters=10,
                            
                            interval = 10, sel_select = True, sel = 1000,
                                            additional_info = {'size':x['size']}
                           
                           
                           
                       
                       )

,axis =1
)


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

## Frozen Lake Q learning evaluatons | version 1

In [196]:

pd.read_csv('QL_FLFLRL_iter1.csv').apply(
lambda x:

Model_evaluations_tests(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                       
                       
                        
                           render=False,user_input=False ,n_iters=20,
                            
                            interval = 100, sel_select = True, sel = 1000
                           
                           
                           
                       
                       )

,axis =1
)

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

0      None
1      None
2      None
3      None
4      None
       ... 
135    None
136    None
137    None
138    None
139    None
Length: 140, dtype: object

## Frozen Lake Q learning evaluatons | version 2

In [197]:

pd.read_csv('QL_FLFLRL_iter2.csv').apply(
lambda x:

Model_evaluations_tests(n_=x['address_name'],
                            env=x['env_name'],algo=x['iter_'],
                       
                       
                        
                           render=False,user_input=False ,n_iters=20,
                            
                            interval = 100, sel_select = True, sel = 1000
                           
                           
                           
                       
                       )

,axis =1
)

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

0      None
1      None
2      None
3      None
4      None
       ... 
975    None
976    None
977    None
978    None
979    None
Length: 980, dtype: object